In [1]:
import numpy as np
import math
import unittest

In [2]:
def is_triangular(a:np.ndarray):
    return np.allclose(a[:, :4], np.triu(a[:, :4]))
def is_singular(a:np.ndarray):
    return np.linalg.det(a)
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    return 0

In [3]:
def reflection_method(a:np.array, b:np.array) -> np.array:
    b = b.copy()
    a = a.copy()
    n, m = a.shape
    for i in range(n-1):
        s = sign(-a[i,i])*np.linalg.norm(a[:, i])
        etta = 1 / math.sqrt(2 * s * (s - a[i,i]))
        w = a[:,i][..., np.newaxis].copy()
        for j in range(i):
            w[j, 0] = 0
        w[i, 0] -= s
        w *= etta
        u = np.eye(n) - 2 * np.matmul(w, w.T)
        a = np.matmul(u, a)
        b = np.matmul(u, b)
    return backward_gauss_method(np.concatenate((a, b), axis=1))

In [4]:
def backward_gauss_method(a:np.array) -> np.array:
    a = a.copy()
    n, m = a.shape
    x = np.zeros((n, 1))
    for k in range(n-1, -1, -1):
        x[k] = (a[k, -1] - np.dot(a[k, k:n], x[k:n])) / a[k, k]
    return x

In [5]:
def rotation_method(a:np.array) -> np.array:
    a = a.copy()
    n, m = a.shape
    for k in range(0, m - 1):
        for j in range(k + 1, n):
            temp = math.sqrt(a[k,k] ** 2 + a[j,k] ** 2)
            cos = a[k,k] / (temp)
            sin = a[j,k] / (temp)
            a_i = cos * a[k] + sin * a[j]
            a_j = -sin * a[k] + cos * a[j]
            a[k] = a_i
            a[j] = a_j
    return backward_gauss_method(a)

In [60]:
def find_inverse_matrix(a_inner:np.array) -> np.array:
    a = a_inner.copy()
    n = a.shape[0]
    a = np.append(a, np.eye(n), 1)
    for k in range(0, n - 1):
        for j in range(k + 1, n):
            temp = math.sqrt(a[k,k] ** 2 + a[j,k] ** 2)
            cos = a[k,k] / (temp)
            sin = a[j,k] / (temp)
            a_i = cos * a[k] + sin * a[j]
            a_j = -sin * a[k] + cos * a[j]
            a[k] = a_i
            a[j] = a_j
    for k in range(n-1, -1, -1):
        a[k, :] /= a[k,k]
        for i in range(k-1, -1, -1):
            a[i, :] -= a[i, k] * a[k, :]
    return a[:, n:]

In [61]:
a = np.array([[5, 4, 3], [0, 5, 6], [0, 0, 1]], dtype='float64')
print(find_inverse_matrix(a))
np.linalg.inv(a)


[[ 0.2  -0.16  0.36]
 [ 0.    0.2  -1.2 ]
 [ 0.    0.    1.  ]]


array([[ 0.2 , -0.16,  0.36],
       [ 0.  ,  0.2 , -1.2 ],
       [ 0.  ,  0.  ,  1.  ]])

In [64]:
import unittest
class TestBackwardGaussMethod(unittest.TestCase):
    def pattern(self, a, x):
        b = np.matmul(a, x)
        a_extend = np.append(a, b, axis=1)
        calc_x = backward_gauss_method(a_extend)
        is_close = np.allclose(
            calc_x,
            x
        )
        self.assertTrue(is_close)
    def test_5_x_5(self):
        a = np.array([
            [1, 2, 3, 4, 5],
            [0, 7, 6, 1, 2],
            [0, 0, 3, 5, 8],
            [0, 0, 0, 9, 2],
            [0, 0, 0, 0, 1]
        ], dtype='float64')
        x = np.array([[-5, 6, 8, -2, -4]]).T
        self.pattern(a, x)
    def test_3x3(self):
        a = np.array([
            [7, -2, 3],
            [0, 9, 0],
            [0, 0, -2]
        ], dtype='float64')
        x = np.array([[-1, 2, -6]]).T
        self.pattern(a, x)
class TestRotationMethod(unittest.TestCase):
    def setUp(self):
        print(type(self).__name__)
    def pattern(self, a, x, method_name):
        b = np.matmul(a, x)
        a_extend = np.append(a, b, axis=1)
        calc_x = rotation_method(a_extend)
        is_close = np.allclose(
            calc_x,
            x
        )
        discrepancy = b - np.matmul(a, calc_x)
        print(str(method_name), '\n', discrepancy)
        self.assertTrue(is_close)
    def test_symmetric(self):
        a = np.array([
            [0.521,-0.296, 0, 0.04, -0.21],
            [-0.296, 0.7, 0.24, -0.32, 0.06],
            [0, 0.24, -0.35, 0.07, 0.21],
            [0.04, -0.32, 0.7, 0.49, -0.03],
            [-0.21, 0.06, 0.21, -0.03, 0.63]
        ], dtype='float64')
        x = np.array([[1, -3, 4, 8, -1]]).T
        self.pattern(a, x, self.test_symmetric)
    def test_square(self):
        a = np.array([
            [2, 3, -4, 1],
            [1, -2, -5, 1],
            [5, -3, 1, -4],
            [10, 2, -1, 2]
        ], dtype='float64')
        x = np.array([[1, 3, -4, 2]], dtype='float64').T
        self.pattern(a, x, self.test_square)
    def test_hilbert(self):
        n = 5
        a = np.array([[1/(i + j - 1) for j in range(1, n+1)] for i in range(1, n + 1)])
        x = np.array([[-5, 6, 8, -2, -4]]).T
        self.pattern(a, x, self.test_hilbert)
# class TestReflectionMethod(unittest.TestCase):
#     def setUp(self):
#         print(type(self).__name__)
#     def pattern(self, a, x, method_name):
#         b = np.matmul(a, x)
#         a_extend = np.append(a, b, axis=1)
#         calc_x = reflection_method(a_extend)
#         is_close = np.allclose(
#             calc_x,
#             x
#         )
#         discrepancy = b - np.matmul(a, calc_x)
#         print(str(method_name), '\n', discrepancy)
#         self.assertTrue(is_close)
#     def test_symmetric(self):
#         a = np.array([
#             [0.521,-0.296, 0, 0.04, -0.21],
#             [-0.296, 0.7, 0.24, -0.32, 0.06],
#             [0, 0.24, -0.35, 0.07, 0.21],
#             [0.04, -0.32, 0.7, 0.49, -0.03],
#             [-0.21, 0.06, 0.21, -0.03, 0.63]
#         ], dtype='float64')
#         x = np.array([[1, -3, 4, 8, -1]]).T
#         self.pattern(a, x, self.test_symmetric)
#     def test_square(self):
#         a = np.array([
#             [2, 3, -4, 1],
#             [1, -2, -5, 1],
#             [5, -3, 1, -4],
#             [10, 2, -1, 2]
#         ], dtype='float64')
#         x = np.array([[1, 3, -4, 2]], dtype='float64').T
#         self.pattern(a, x, self.test_square)
#     def test_hilbert(self):
#         n = 5
#         a = np.array([[1/(i + j - 1) for j in range(1, n+1)] for i in range(1, n + 1)])
#         x = np.array([[-5, 6, 8, -2, -4]]).T
#         self.pattern(a, x, self.test_hilbert)
class TestInvMatrix(unittest.TestCase):
    def pattern(self, a):
        inv = find_inverse_matrix(a)
        self.assertTrue(np.allclose(inv, np.linalg.inv(a)))
        print(np.linalg.norm(a) * np.linalg.norm(inv))
    def test_symmetric(self):
        a = np.array([
            [0.521,-0.296, 0, 0.04, -0.21],
            [-0.296, 0.7, 0.24, -0.32, 0.06],
            [0, 0.24, -0.35, 0.07, 0.21],
            [0.04, -0.32, 0.7, 0.49, -0.03],
            [-0.21, 0.06, 0.21, -0.03, 0.63]
        ], dtype='float64')
        print("Inv symmetric")
        self.pattern(a)
    def test_square(self):
        a = np.array([
            [2, 3, -4, 1],
            [1, -2, -5, 1],
            [5, -3, 1, -4],
            [10, 2, -1, 2]
        ], dtype='float64')
        print("Inv square")
        self.pattern(a)
    def test_hilbert(self):
        n = 5
        a = np.array([[1/(i + j - 1) for j in range(1, n+1)] for i in range(1, n + 1)])
        print("Inv hilbert")
        self.pattern(a)
unittest.main(argv=[''], verbosity=2, exit=False)

test_3x3 (__main__.TestBackwardGaussMethod) ... ok
test_5_x_5 (__main__.TestBackwardGaussMethod) ... ok
test_hilbert (__main__.TestInvMatrix) ... ok
test_square (__main__.TestInvMatrix) ... ok
test_symmetric (__main__.TestInvMatrix) ... ok
test_hilbert (__main__.TestReflectionMethod) ... ERROR
test_square (__main__.TestReflectionMethod) ... ERROR
test_symmetric (__main__.TestReflectionMethod) ... ERROR
test_hilbert (__main__.TestRotationMethod) ... ok
test_square (__main__.TestRotationMethod) ... ok
test_symmetric (__main__.TestRotationMethod) ... ok

ERROR: test_hilbert (__main__.TestReflectionMethod)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-62-502558c9c1a7>", line 105, in test_hilbert
    self.pattern(a, x, self.test_hilbert)
  File "<ipython-input-62-502558c9c1a7>", line 74, in pattern
    calc_x = reflection_method(a_extend)
TypeError: reflection_method() missing 1 required positional argument: 

Inv hilbert
480849.1169946468
Inv square
7.544174792155575
Inv symmetric
8.70100117809442
TestReflectionMethod
TestReflectionMethod
TestReflectionMethod
TestRotationMethod
<bound method TestRotationMethod.test_hilbert of <__main__.TestRotationMethod testMethod=test_hilbert>> 
 [[1.33226763e-15]
 [6.66133815e-16]
 [5.55111512e-16]
 [4.99600361e-16]
 [2.22044605e-16]]
TestRotationMethod
<bound method TestRotationMethod.test_square of <__main__.TestRotationMethod testMethod=test_square>> 
 [[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-3.55271368e-15]
 [ 0.00000000e+00]]
TestRotationMethod
<bound method TestRotationMethod.test_symmetric of <__main__.TestRotationMethod testMethod=test_symmetric>> 
 [[-8.88178420e-16]
 [ 0.00000000e+00]
 [-6.66133815e-16]
 [ 0.00000000e+00]
 [-1.66533454e-16]]
